In [33]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

was trying an amazon sagemaker just to get more compute power for training the model. this is left over code from that.

In [34]:
# import zipfile
# import os

# def unzip_file(zip_file_path):
#     # Get the directory where the zip file is located
#     extract_to_directory = os.path.splitext(zip_file_path)[0]

#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to_directory)

# notebook_directory = os.path.dirname(os.path.abspath("main.ipynb"))

# # Replace 'Kindle_Store.zip' with the actual name of your zip file
# zip_file_name = 'Kindle_Store.zip'

# # Full path to the zip file
# zip_file_path = os.path.join(notebook_directory, zip_file_name)
# print(zip_file_path)
# # Check if the zip file exists
# if os.path.exists(zip_file_path):
#     # Call the function to unzip the file
#     unzip_file(zip_file_path)
# else:
#     print("The specified zip file does not exist.")


### Loading the data using chunking

In [35]:
# Read JSON file in chunks
chunk_size = 100000  # Adjust the chunk size as needed
chunks = []
with open('Kindle_Store.json', 'r') as file:
    for chunk in pd.read_json(file, chunksize=chunk_size, lines=True):
        chunks.append(chunk)

# Concatenate all chunks to form a single DataFrame
df = pd.concat(chunks, ignore_index=True)

In [36]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"12 29, 2012",A27UD5HYAKBL97,1423600150,{'Format:': ' Hardcover'},Cheryl,If you like making salsas this is a great book...,Great Book,1356739200,NaN,NaN
1,5,True,"03 6, 2012",A8P5DK8LLOYGH,1423600150,{'Format:': ' Hardcover'},Shay365,great little book. simple and right to the poi...,great little book,1330992000,3,NaN
2,5,True,"08 12, 2009",A3OM9W7DXSUIIY,1423600150,{'Format:': ' Hardcover'},R. Peckham,This book has good pics of the recipes and eas...,very good bok with good ideas.,1250035200,NaN,NaN
3,3,False,"11 3, 2006",A3DPAR2PWB9BT8,1423600150,{'Format:': ' Hardcover'},JJSS,"I admire the SF School of Cooking, but was dis...",Ok cookbook,1162512000,3,NaN
4,5,False,"04 11, 2006",A2ONTBSAX7GIOV,1423600150,{'Format:': ' Hardcover'},Three Guys from Miami,Many people are surprised to find out that we ...,Fresh and Tasty Ideas from Santa Fe!,1144713600,15,NaN


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5722988 entries, 0 to 5722987
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   overall         int64 
 1   verified        bool  
 2   reviewTime      object
 3   reviewerID      object
 4   asin            object
 5   style           object
 6   reviewerName    object
 7   reviewText      object
 8   summary         object
 9   unixReviewTime  int64 
 10  vote            object
 11  image           object
dtypes: bool(1), int64(2), object(9)
memory usage: 485.8+ MB


In [38]:
# if image is non null return 1 else 0
df['image'] = df['image'].notnull().astype('int')

In [39]:
# remove commas from vote column
df['vote'] = df['vote'].str.replace(',', '')
# cast vote column to int, converting Nan to 0
df['vote'] = df['vote'].fillna(0)
df['vote'] = df['vote'].astype('int')

In [40]:
# in review text replace null values with empty string
df['reviewText'] = df['reviewText'].fillna('')

In [41]:
# convert verified to int 1 if true 0 if false
df['verified'] = df['verified'].astype('int')

In [47]:
vectorizer = CountVectorizer(stop_words='english')
reviews = df['reviewText'].tolist()
fitted = vectorizer.fit(reviews)
bow = vectorizer.transform(reviews)

In [48]:
# spmatrix to dataframe
bow_df = pd.DataFrame.sparse.from_spmatrix(bow)
bow_df.columns = bo.get_feature_names()
bow_df.head()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Step 1: Prepare the data
X = bow  # Your Bag-of-Words representation
y = df['overall']  # Target variable

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train a logistic regression model
log_reg = LogisticRegression(max_iter=1000)  # You can tune hyperparameters as needed
log_reg.fit(X_train, y_train)

# Step 4: Evaluate the model
accuracy = log_reg.score(X_test, y_test)
print(f"Accuracy of the logistic regression model: {accuracy:.2f}")


KeyboardInterrupt: 